In [1]:
import pandas as pd

# Đọc file
df = pd.read_csv("car.csv")

# Xem qua
df.head()
print(f"Số dòng: {df.shape[0]}")





Số dòng: 317636


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317636 entries, 0 to 317635
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                317636 non-null  int64  
 1   list_id           317636 non-null  int64  
 2   list_time         317636 non-null  int64  
 3   manufacture_date  317636 non-null  int64  
 4   brand             317586 non-null  object 
 5   model             317586 non-null  object 
 6   origin            234317 non-null  object 
 7   type              272934 non-null  object 
 8   seats             283792 non-null  float64
 9   gearbox           316243 non-null  object 
 10  fuel              317542 non-null  object 
 11  color             221540 non-null  object 
 12  mileage_v2        317636 non-null  int64  
 13  price             317524 non-null  float64
 14  condition         317636 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 36.4+ MB


In [3]:
# Tính số lượng và tỉ lệ missing values
missing_counts = df.isnull().sum()
missing_percent = df.isnull().mean() * 100

# Gộp thành bảng dễ đọc
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_percent
}).sort_values(by='Missing %', ascending=False)

missing_df


,Missing Count,Missing %
color,96096,30.253498
origin,83319,26.230969
type,44702,14.073342
seats,33844,10.654964
gearbox,1393,0.438552
price,112,0.035260
fuel,94,0.029594
brand,50,0.015741
model,50,0.015741
id,0,0.000000


In [4]:
# Danh sách các cột bắt buộc không được null
required_cols = ['gearbox', 'price', 'fuel', 'brand', 'model']

# Loại bỏ dòng thiếu bất kỳ cột nào trong danh sách
df = df.dropna(subset=required_cols)

# Kiểm tra lại sau khi loại
print(f"Số dòng còn lại sau khi loại: {df.shape[0]}")
df.info()


Số dòng còn lại sau khi loại: 316037
<class 'pandas.core.frame.DataFrame'>
Index: 316037 entries, 0 to 317635
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                316037 non-null  int64  
 1   list_id           316037 non-null  int64  
 2   list_time         316037 non-null  int64  
 3   manufacture_date  316037 non-null  int64  
 4   brand             316037 non-null  object 
 5   model             316037 non-null  object 
 6   origin            233266 non-null  object 
 7   type              271451 non-null  object 
 8   seats             282298 non-null  float64
 9   gearbox           316037 non-null  object 
 10  fuel              316037 non-null  object 
 11  color             220477 non-null  object 
 12  mileage_v2        316037 non-null  int64  
 13  price             316037 non-null  float64
 14  condition         316037 non-null  object 
dtypes: float64(2), int64(5), object(8)
m

In [5]:
missing_counts = df.isnull().sum()
missing_percent = df.isnull().mean() * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_percent
}).sort_values(by='Missing %', ascending=False)

missing_df


,Missing Count,Missing %
color,95560,30.236966
origin,82771,26.190288
type,44586,14.107842
seats,33739,10.675649
id,0,0.000000
list_id,0,0.000000
list_time,0,0.000000
manufacture_date,0,0.000000
brand,0,0.000000
model,0,0.000000


In [6]:
# 1. Tạo bảng mode của seats theo brand + type
mode_seats = (
    df[df['seats'].notnull()]
    .groupby(['brand', 'type'])['seats']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    .reset_index()
    .rename(columns={'seats': 'mode_seats'})
)

# 2. Gộp bảng này vào df gốc
df = df.merge(mode_seats, on=['brand', 'type'], how='left')

# 3. Nếu seats bị thiếu → điền bằng mode_seats
df['seats'] = df['seats'].fillna(df['mode_seats'])

# 4. Bỏ cột phụ nếu không cần
df = df.drop(columns=['mode_seats'])


In [7]:
print("Số dòng còn thiếu seats sau khi điền:", df['seats'].isnull().sum())


Số dòng còn thiếu seats sau khi điền: 29842


In [8]:
# Chuyển list_time từ milliseconds → datetime
df['list_time'] = pd.to_datetime(df['list_time'], unit='ms', errors='coerce')
#sort
df = df.sort_values(by='list_time', ascending=True)


In [9]:
df['record_id'] = range(1, len(df) + 1)


In [10]:
cols = ['record_id'] + [col for col in df.columns if col != 'record_id']
df = df[cols]


In [11]:
df['image_path'] = df['record_id'].apply(lambda x: f"images/record_{int(x)}.jpg")


In [ ]:
import os

def get_image_path(record_id):
    path = f"images/{int(record_id)}.jpg"
    return path if os.path.exists(path) else "images/default.jpg"

df['image_path'] = df['record_id'].apply(get_image_path)


In [12]:
df.to_csv("cleaned_data_sort.csv", index=False)
